# AD - Write HDF5 Files Using Single Mode

**Objective**

The [EPICS area detector](https://areadetector.github.io/master/index.html) software has a `Single` mode that reduces the configuration steps needed to acquire an image frame (?**or frames**?) and output a file.  Here, we show how to configure the EPICS controls, then acquire an image with [bluesky](https://blueskyproject.io/) and write it to an [HDF5](https://www.hdfgroup.org/solutions/hdf5) file.

**Contents**

- [EPICS Area Detector IOC](#EPICS-Area-Detector-IOC) is pre-built
- [File Directories](#File-Directories) are different on IOC and bluesky workstation
- [ophyd](#ophyd) to describe the hardware
- [bluesky](#bluesky) for the measurement
- [databroker](#databroker) to view the image
- [punx](#punx) (not part of Bluesky) to look at the HDF5 file
- [Recapitulation](#Recapitulation) - rendition with no explanations

In other examples (such as AD with [default file names](./de_adsim_hdf5_basic.ipynb) or [custom file names](./de_adsim_hdf5_custom_names.ipynb)), we described the details of the area detector support.  Refer to those examples for the details.  Here, following the same general outline, we leverage that knowledge and proceed to the specifics for this example.

***NOTE***

This might need some customizations for `ophyd.areadetector.filestore_mixins.FileStoreHDF5IterativeWrite` to configure the staging properly and not include the `det.hdf1.capture` signal.  Also need to ensure `point_number` is written properly for databroker to retrieve the image from the HDF5 file.

Implementation here should consider [#671](https://github.com/BCDA-APS/apstools/issues/671). The [NSLS-II area detector support](https://github.com/NSLS-II/nslsii/blob/9cd77e0aa24d59fddf534948b9b97dacb0832ed2/nslsii/ad33.py#L37) has some changes.  Here, new version-specific mixins would be appropriate.

## EPICS Area Detector IOC

This example uses a prebuilt [ADSimDetector](https://areadetector.github.io/master/ADSimDetector/simDetector.html) driver, packaged in a [docker](https://www.docker.com/) image
([prjemian/custom-synapps-6.2-ad-3.10](https://hub.docker.com/r/prjemian/custom-synapps-6.2-ad-3.10/tags)).  The [EPICS IOC](https://docs.epics-controls.org/projects/how-tos/en/latest/getting-started/creating-ioc.html) is configured with prefix `ad:` using the [bash shell script](https://github.com/prjemian/epics-docker/blob/main/v1.1/n6_custom_areaDetector/start_adsim.sh):

```bash
user@workstation:~$ start_adsim.sh ad
```

In [ ]:
IOC = "ad:"

## File Directories

Files from the IOC are mounted on the docker host in the directory `/tmp/docker_ioc/iocad`. The bluesky session runs on the docker host.

system | file directory
--- | ---
area detector IOC | `/tmp`
bluesky | `/tmp/docker_ioc/iocad/tmp`

In [ ]:
import pathlib

AD_IOC_MOUNT_PATH = pathlib.Path("/tmp")
BLUESKY_MOUNT_PATH = pathlib.Path("/tmp/docker_ioc/iocad/tmp")

## ophyd

Here's a screen view of the configuration we want (`HDF` plugin on the left,
`cam` plugin on the right):

![Area Detector configuration for writing HDF5 files in Single mode](ad_hdf5_single_mode.png)

### cam

The `cam` Device describes the EPICS area detector camera driver for this detector.

TODO

### HDF5

The `hdf1` Device describes the HDF5 File Writing plugin for this detector.

TODO

### detector

The detector class, a subclass of `DetectorBase`, brings together the detector driver `cam` and plugins.

TODO

With all the above setup, create the Python detector object, `adsimdet` and wait for it to connect with EPICS.

In [ ]:
adsimdet = TheDetector(IOC, name="adsimdet")
adsimdet.wait_for_connection(timeout=15)

Check that all plugins used by the IOC have been defined in the Python structure.  Expect that this function returns an empty list: `[]`.

In [ ]:
adsimdet.missing_plugins()

We must configure `adsimdet` so the HDF5 plugin (by its attribute name `hdf1`) will be called during `adsimdet.read()`, as used by data acquisition.

In [ ]:
adsimdet.read_attrs.append("hdf1")

Configure the HDF5 plugin so it will create up to 5 subdirectories for the image directory.

In [ ]:
adsimdet.hdf1.create_directory.put(-5)

*Prime* the HDF5 plugin, if necessary.

In [ ]:
from apstools.devices import AD_plugin_primed
from apstools.devices import AD_prime_plugin2

# this step is needed for ophyd
if not AD_plugin_primed(adsimdet.hdf1):
    print(f"Priming {adsimdet.hdf1.dotted_name}")
    AD_prime_plugin2(adsimdet.hdf1)

## bluesky

Within the [Bluesky framework](https://blueskyproject.io/), [bluesky](https://blueskyproject.io/bluesky) is the package that orchestrates the data acquisition steps, including where to direct acquired data for storage.  [Later](#databroker), we'll use [databroker](https://blueskyproject.io/databroker) to access the image data.

TODO

We'll use a temporary databroker catalog for this example and setup the RunEngine object `RE`.

TODO

***Take an image with the area detector***

TODO

In [ ]:
uids = RE(bp.count([adsimdet], md=dict(title="Area Detector with HDF5 plugin", purpose="image")))

## databroker



TODO

In [ ]:
run = cat.v2[uids[0]]
run

***Get the image frame from the run***

From the run, we know the image data is in the primary stream.

TODO

In [ ]:
# import hdf5plugin  # required for LZ4, Blosc, and other compression codecs

dataset = run.primary.read()
dataset

The image is recorded under the name `"adsimdet_image"`.

TODO

In [ ]:
image = dataset["adsimdet_image"]
# image is an xarray.DataArray with 1 timestamp and 5 frames of 1k x 1k

We just want the `image` frame (the last two indices).

TODO

In [ ]:
frame = image[0][0]
# frame is an xarray.DataArray of 1k x 1k


***Visualize the image***

The `frame` is an [xarray Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html), which has a method to visualize the data as shown here:

TODO

In [ ]:
frame.plot.pcolormesh()

In [ ]:
run.primary._resources

***Find the image file on local disk***

Get the name of the image file on the bluesky (local) workstation from the `adsimdet` object.

In [ ]:
from apstools.devices import AD_full_file_name_local

local_file_name = AD_full_file_name_local(adsimdet.hdf1)
print(f"{local_file_name.exists()=}\n{local_file_name=}")

Alternatively, we might get the name of the file from the run stream.

In [ ]:
rsrc = run.primary._resources[0]
fname = pathlib.Path(f"{rsrc['root']}{rsrc['resource_path']}")
print(f"{fname.exists()=}\n{fname=}")

# confirm they are the same
print(f"{(local_file_name == fname)=}")

## punx

Next, we demonstrate access to the HDF5 image file using the [punx](https://punx.readthedocs.io) program.

TODO

In [ ]:
from apstools.utils import unix

for line in unix(f"punx tree {local_file_name}"):
    print(line.decode().strip())

## Recapitulation

Let's gather the above parts together as one would usually write code.  First, all the imports, constants, and classes.

TODO